## Define Constants

In [1]:
PROJECT_ID = 'jchavezar-demo'
TRAINING_IMAGE_URI = f'gcr.io/{PROJECT_ID}/demos-train-azure:latest'
SERVING_CONTAINER_IMAGE_URI = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-9:latest'
DATASET_NAME = 'fraud_detection.csv'
MODEL_DIR = 'gs://vtx-models/azure/hpt'

## Create Folder Structure

In [2]:
import os

if not os.path.exists("1_custom_train_job"):
    os.makedirs("1_custom_train_job")

## Create Training File

In [3]:
%%writefile 1_custom_train_job/main.py

# Extracting information from Azure blob storage

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

FILENAME = os.environ['FILE_NAME']
connect_str = os.environ['AZURE_STORAGE_CONNECTION_STRING']
print(FILENAME)
print(connect_str)
    
# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
blob_client_instance = blob_service_client.get_blob_client('vertex', FILENAME, snapshot=None)
    
with open(FILENAME, 'wb') as my_blob:
    blob_data = blob_client_instance.download_blob()
    blob_data.readinto(my_blob)
    
df = pd.read_csv(FILENAME)

train_df = df.sample(frac=0.8, random_state=1)
test_df = df.drop(train_df.index)

X_train = train_df.iloc[:,:-1]
X_test = test_df.iloc[:,:-1]
y_train = train_df['Class'].astype(np.float32)
y_test = test_df['Class'].astype(np.float32)

# Standarization

X_train_norm = (X_train-X_train.mean())/X_train.std()
X_test_norm = (X_test-X_test.mean())/X_test.std()
        
# Model

def create_model(my_learning_rate, ds_length):
    from tensorflow.keras import layers
    
    METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]
    
    model = tf.keras.models.Sequential([
        layers.Dense(16, activation='relu', input_shape=[ds_length]),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
        
    optimizer=tf.keras.optimizers.Adam(learning_rate=my_learning_rate)
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=optimizer,
        metrics=METRICS)
    
    return model

def train_model(x, y, model, epochs,
                batch_size=None, shuffle=True):
    history = model.fit(x=x, y=y, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle)
    epochs = history.epoch
    hist = pd.DataFrame(history.history)
    return epochs, hist

# The following variables are the hyperparameters.
learning_rate = 0.001
epochs = 20
batch_size = 100
ds_length = len(X_train_norm.keys())

# Establish the model's topography.
my_model = create_model(learning_rate, ds_length)

# Train the model on the training set.
epochs, hist = train_model(X_train_norm, y_train, my_model, epochs, 
                           batch_size)

my_model.save(os.environ['AIP_MODEL_DIR'])

Overwriting 1_custom_train_job/main.py


## Create Dockerfile

In [4]:
%%writefile 1_custom_train_job/Dockerfile

FROM tensorflow/tensorflow

RUN pip install azure-storage-blob azure-identity pandas

COPY main.py /main.py

CMD ["python", "/main.py"]

Overwriting 1_custom_train_job/Dockerfile


## Create Docker Image with CloudBuild

In [5]:
!gcloud builds submit -t $TRAINING_IMAGE_URI 1_custom_train_job/.

Creating temporary tarball archive of 2 file(s) totalling 2.9 KiB before compression.
Uploading tarball of [1_custom_train_job/.] to [gs://jchavezar-demo_cloudbuild/source/1672238210.184197-7b5495b7ef7d408495d97698bddc063c.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jchavezar-demo/locations/global/builds/e75ba77d-a8dc-456b-b58e-13d02db7d027].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/e75ba77d-a8dc-456b-b58e-13d02db7d027?project=569083142710 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "e75ba77d-a8dc-456b-b58e-13d02db7d027"

FETCHSOURCE
Fetching storage object: gs://jchavezar-demo_cloudbuild/source/1672238210.184197-7b5495b7ef7d408495d97698bddc063c.tgz#1672238210393469
Copying gs://jchavezar-demo_cloudbuild/source/1672238210.184197-7b5495b7ef7d408495d97698bddc063c.tgz#1672238210393469...
/ [1 files][  1.4 KiB/  1.4 KiB]                                                
Operation complet

## Create the Pipeline

In [6]:
import os
import env
from kfp.v2.dsl import pipeline
from google_cloud_pipeline_components.v1.custom_job import CustomTrainingJobOp
from google_cloud_pipeline_components.aiplatform import (
    EndpointCreateOp,
    ModelDeployOp,
    ModelUploadOp,
)

worker_pool_specs = [
    {
        'machineSpec': {
            'machineType': 'n1-standard-4',
    },
        'replicaCount': 1,
        'containerSpec': {
            'image_uri': TRAINING_IMAGE_URI,
            'env': [
                {
                    'name': 'FILE_NAME',
                    'value': DATASET_NAME
                },
                {
                    'name': 'AZURE_STORAGE_CONNECTION_STRING',
                    'value': env.AZURE_STORAGE_CONNECTION_STRING
                },
            ]
        }
    }
]




@pipeline(name='azure-gcp-test')
def pipeline(
    project_id: str,
    model_dir: str,
    serving_image_uri: str
):
    custom_train_task = CustomTrainingJobOp(
        project=project_id,
        display_name='custom_train_task',
        worker_pool_specs = worker_pool_specs,
        base_output_directory=model_dir
    )
    
    model_upload_task = ModelUploadOp(
        project=project_id,
        display_name=f'tf-kfp-prebuilt-model-upload-job',
        artifact_uri=f'{model_dir}/model',
        serving_container_image_uri=serving_image_uri,
    ).after(custom_train_task)
    
    create_endpoint_task = EndpointCreateOp(
        project=project_id,
        display_name='fraud_det'
    ).after(model_upload_task)
    
    model_deploy_task = ModelDeployOp(
        endpoint=create_endpoint_task.outputs["endpoint"],
        model=model_upload_task.outputs["model"],
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
        dedicated_resources_machine_type='n1-standard-4'
    )

## Compile Pipeline

In [7]:
from kfp.v2 import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='azure_gcp_test.json')

/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


## Run Pipeline Job

In [8]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name='customjob-azure-gcp',
    template_path='azure_gcp_test.json',
    pipeline_root='gs://vtx-path-root',
    parameter_values={
        'project_id': 'jchavezar-demo',
        'model_dir': MODEL_DIR,
        'serving_image_uri': SERVING_CONTAINER_IMAGE_URI
    },
    enable_caching=False
)

job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/569083142710/locations/us-central1/pipelineJobs/azure-gcp-test-20221228143801
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/569083142710/locations/us-central1/pipelineJobs/azure-gcp-test-20221228143801')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/azure-gcp-test-20221228143801?project=569083142710


![azure](../images/azure-pipe.png)